In [ ]:
!pip install transformers

In [ ]:
!pip install tensorflow_datasets

In [ ]:
import tensorflow.keras.models

In [ ]:
from transformers import BertTokenizer, TFBertModel, GPT2Tokenizer, TFGPT2Model
import numpy as np
from tensorflow import keras
from tensorflow_datasets import load
import tensorflow_datasets as tfds
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Concatenate, LSTM, Embedding, TimeDistributed
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder, LabelEncoder


In [ ]:
X_img_train = ("https://www.pexels.com/photo/elephants-on-brown-mountain-631317/", "https://www.google.com/imgres?imgurl=https%3A%2F%2Fwallpapers.com%2Fimages%2Fhd%2Fnational-geographic-predator-silhouette-4z13g83xxc7xn5x3.jpg&tbnid=VzNU4MX1n3FYVM&vet=12ahUKEwjM79v-zvuDAxXAcGwGHSLZCjYQMygNegQIARBv..i&imgrefurl=https%3A%2F%2Fwallpapers.com%2Fnational-geographic&docid=7ky-x1XgTMZIrM&w=1920&h=1270&q=geographic%20hd%20pics&ved=2ahUKEwjM79v-zvuDAxXAcGwGHSLZCjYQMygNegQIARBv")
image_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2, height_shift_range=0.2)

In [ ]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
max_len=30
X_bert_train=[]
for input in text_sequences:
    tokens = bert_tokenizer(input, padding='max_length', truncation=True, max_length=max_len, return_tensors='np')
    X_bert_train.append(tokens['input_ids'])

X_bert_train = np.array(X_bert_train)

bert_model = TFBertModel.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
max_len_gpt2=30
X_gpt2_train=[]
gpt2_tokenizer.add_special_tokens({'pad_token': '[PAD]'})
for input in text_sequences:
    tokens = gpt2_tokenizer(input, padding='max_length', truncation=True, max_length=max_len_gpt2, return_tensors='np')
    X_gpt2_train.append(tokens['input_ids'])
X_gpt2_train = np.array(X_gpt2_train)

gpt2_model = TFGPT2Model.from_pretrained('gpt2')


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['qa_outputs.bias', 'qa_outputs.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.
All PyTorch model weights were used when initializing TFGPT2Model.

All the weights of TFGPT2Model were initialized from the PyTorch model.
If your task is similar to the task the model of th

In [ ]:
import tensorflow

In [ ]:
image_input = Input(shape=(299, 299, 3))
image_features = Conv2D(64, (3, 3), activation='relu')(image_input)
image_features = MaxPooling2D(pool_size=(2, 2))(image_features)
image_features = Flatten()(image_features)
image_features = Dense(256, activation='relu')(image_features)
vocab_size=10000
X_seq_train= ["how many elephants", "color of elephants"]
text_input = Input(shape=(max_len,))
text_features = Embedding(input_dim=vocab_size, output_dim=256, input_length=max_len)(text_input)
text_features = LSTM(256, return_sequences=False)(text_features)
bert_input = Input(shape=(max_len,), dtype='int32')
bert_output= bert_model(bert_input)['pooler_output']
bert_features =Dense(256, activation='relu')(bert_output) # [1] returns the pooled output
gpt2_input = Input(shape=(max_len,))
gpt2_features = gpt2_model(gpt2_input)[0][:, 0, :]  # Using the CLS token representation

concatenated_features = Concatenate()([image_features, text_features, bert_features, gpt2_features])

output = Dense(vocab_size, activation='softmax')(concatenated_features)

model = Model(inputs=[image_input, text_input, bert_input, gpt2_input], outputs=output)


In [ ]:
optimizer = Adam(learning_rate=0.001)  # Adjust learning rate based on experimentation
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


In [ ]:
label_encoder= LabelEncoder()
labels=["12", "black"]
y_train = label_encoder.fit_transform(labels)
y_train_one_hot= to_categorical(y_train)

In [ ]:
epochs=10
batch_size=32

In [ ]:
# Assuming you have your augmented image and question data ready in X_img_train, X_seq_train, and y_train
model.fit([X_img_train, X_seq_train, X_bert_train, X_gpt2_train], y_train, epochs = epochs, batch_size=batch_size)


ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {'(<class \'tuple\'> containing values of types {"<class \'str\'>"})', '(<class \'list\'> containing values of types {"<class \'str\'>"})', "<class 'numpy.ndarray'>"}), <class 'numpy.ndarray'>

In [ ]:
print(type(X_img_train))
print(type(X_seq_train))
print(type(X_bert_train))
print(type(X_gpt2_train))
print(type(y_train))

<class 'list'>
<class 'list'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [ ]:
print(X_img_train.shape)
print(X_seq_train.shape)
print(X_bert_train.shape)
print(X_gpt2_train.shape)
print(y_train.shape)

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 conv2d_1 (Conv2D)           (None, 297, 297, 64)         1792      ['input_5[0][0]']             
                                                                                                  
 max_pooling2d_1 (MaxPoolin  (None, 148, 148, 64)         0         ['conv2d_1[0][0]']            
 g2D)                                                                                             
                                                                                                  
 input_6 (InputLayer)        [(None, 30)]                 0         []                      